In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import boto3
import s3fs
from ipywidgets import interact, interactive, fixed, interact_manual,Output
import ipywidgets as ipw
from IPython.display import clear_output,HTML


In [2]:
df = []
odf = []
fdf = []
df2 = []
tf =[]
errordf = []
interval = int
out = Output()
pd.options.mode.chained_assignment = None

def logSearch():#Read log and save as df and odf.
    global df,odf
    out.clear_output()
    df=[]
    odf=[]
    if len(cname)>0 and len(player)>0:
        s3 = boto3.resource('s3')
        bucket = s3.Bucket('s3logbucket-testing')
        for obj in bucket.objects.filter(Prefix='%s/%s/log_%s'%(cname,player,logdate)):
            data = pd.read_json('s3://s3logbucket-testing/'+obj.key)
            df.append(data)
        if len(df) == 0:
            with out:
                print('Invalid input, please try again')                
        else:
            odf = errordf = df
            timeSplit()
    else:
        with out:
            print('company name and player name are required')

def intOnly(message,defaultVal):#Input for time interval return input
    Input = input(message)
    if Input == '':
        Input = int(defaultVal)
    else:
        try:
            Input = int(Input)
        except ValueError:
            print('Input must be integer!')
            intOnly(message,defaultVal)            
    if Input < 0:
        print('Input cannot be negative!')
        intOnly(message,defaultVal)
    return int(Input)

def logFilter():#filter columns then row save filtered dataframe in df*
    global df,odf,fdf,col,logfil
    out.clear_output()
    temp = 0
    fdf.clear()
    if len(col) > 0:
        if len(logfil) > 0:
            if col == 'className':
                for x in range(len(odf)):
                    fdf.append(odf[x][odf[x].className.str.contains(logfil)])
                    fdf[x].index = np.arange(1, len(fdf[x]) + 1)
                    temp += len(fdf[x])
            elif col == 'description':
                for x in range(len(odf)):
                    fdf.append(odf[x][odf[x].description.str.contains(logfil)])
                    fdf[x].index = np.arange(1, len(fdf[x]) + 1)
                    temp += len(fdf[x])
            elif col == 'functionName':
                for x in range(len(odf)):
                    fdf.append(odf[x][odf[x].functionName.str.contains(logfil)])
                    fdf[x].index = np.arange(1, len(fdf[x]) + 1)
                    temp += len(fdf[x])
            elif col == 'logDateTime':
                for x in range(len(odf)):
                    fdf.append(odf[x][odf[x].logDateTime.str.contains(logfil)])
                    fdf[x].index = np.arange(1, len(fdf[x]) + 1)
                    temp += len(fdf[x])
            elif col == 'logType':
                for x in range(len(odf)):
                    fdf.append(odf[x][odf[x].logType.str.contains(logfil)])
                    fdf[x].index = np.arange(1, len(fdf[x]) + 1)
                    temp += len(fdf[x])
            if temp == 0:
                with out:
                    print('No result, please try again!')
                    temp = 0
            else:
                df = fdf
                with out:                    
                    print('Filter succesfully')
        else:
            with out:
                print('Please enter row filter!')

def timeSplit():
    global df
    for x in range(len(df)):
        df[x]['logDateTime'] = df[x]['logDateTime'].str.split("_")
        df[x].index = np.arange(1, len(df[x]) + 1)
        #Filter onStatusChanged

def disMerge():
    global df
    for x in range(len(errordf)):
        df[x].loc[df[x]['description'].str.contains("Reconnecting"),'description'] = 'Reconnecting'
        df[x].loc[df[x]['description'].str.contains("Connecting"),'description'] = 'Reconnecting'
        df[x].loc[df[x]['description'].str.contains("ConnectionLost"),'description'] = 'Reconnecting'
        df[x].loc[df[x]['description'].str.contains("Keep"),'description'] = 'Connected'

def plotError():
    global df,interval,tf
    tf = []
    minframe = []
    hrframe = []
    inc,zero = 0,0
    seq = int(1440/interval)  
    stop = interval - 1
    for x in range(len(df)):
        tf.append([])
        for y in range(seq):
            tf[x].append(hrframe)
            for z in range(1,len(df[x])+1):
                if len(tf[x][y]) == 0:
                    if y == 0:
                        if (0 <= int(df[x]['logDateTime'][z][5]) <= stop) and (int(df[x]['logDateTime'][z][4]) == 0):
                            minframe.append(df[x]['logDateTime'][z])
                            tf[x][y].append(minframe)
                            minframe = []
                    else:
                        start = divmod(y*interval,60)[1]
                        if (start <= int(df[x]['logDateTime'][z][5]) <= start+stop) and (int(df[x]['logDateTime'][z][4]) == int(y*(24/seq))):
                            minframe.append(df[x]['logDateTime'][z])
                            inc+=interval                        
                            tf[x][y].append(minframe)
                            minframe = []
                else:
                    break
            hrframe = []
            


In [27]:
def getLogVal(b):
    global cname,cnamebox,player,playerbox,logdate,logdatebox 
    cname = cnamebox.value
    player = playerbox.value
    logdate = logdatebox.value
    logSearch()
def getFilterVal(b):
    global col,logfil
    col = logcol.value
    logfil = logrow.value
    logFilter()
def getErrorTf(b):
    global interval
    interval = errorTf.value
    out.clear_output()
    disMerge()
    plotError()
    plotdis()

In [28]:
cname = ''
player = ''

cnamebox = ipw.Text(value='company-a',
                    placeholder='company name',
                    description='company:',
                    disabled=False)
playerbox = ipw.Text(value='01',
                    placeholder='player name',
                    description='player:',
                    disabled=False)
logdatebox = ipw.Text(value='01',
                    placeholder='log name not required',
                    description='log:')
readButton = ipw.Button(
    description='Submit',
    disabled=False,
    button_style='',
    tooltip='Click me'
)    
readButton.on_click(getLogVal)

In [29]:
logcol = ipw.Dropdown(
        options=[('Class', 'className'), ('Description', 'description'), ('Function', 'functionName'),
                 ('Date/Time','logDateTime'),('Type','logType')],
        value='functionName',
        description='Column:',
        disabled=False,
    )
logrow = ipw.Text(value='onStatusChanged',
                    placeholder='row filter',
                    description='Row:',
                    disabled=False)
filterButton = ipw.Button(
    description='filter',
    disabled=False,
    button_style='',
    tooltip='Click me'
)    

filterButton.on_click(getFilterVal)

In [30]:
showDf = ipw.Button(
    description = 'Show log',
    disabled = False,
    tooltip = 'Click me'
)
def showdf(b):
    global df
    out.clear_output()
    if len(df)>0:
        for x in range(len(df)):
            with out:
                print(df[x])
    else:
        with out:
            print('No DataFrame to show!')
showDf.on_click(showdf)

In [31]:
plotDis = ipw.Button(
    description = 'Plot disconnection',
    disabled = False
)

def plotdis():
    global tf
    if len(tf)>0:
        for x in range(len(tf)):
            with out:
                print(tf[x])
    else:
        with out:
            print('No DataFrame to show!')
    
plotDis.on_click(getErrorTf)

In [32]:
errorTf = ipw.Dropdown(
    options = [('30 mins',30),('1 hr',60)],
    value = 60,
    description = 'Timeframe',
    disable = False
    )

def plotErrorButton(b):
    global df,out
    out.clear_output()
    

In [33]:
readSec = ipw.VBox([cnamebox,playerbox,logdatebox,readButton])
filterSec = ipw.VBox([logcol,logrow,filterButton])
sec1 = ipw.HBox([readSec,filterSec])
sec2 = ipw.HBox([showDf])
sec3 = ipw.HBox([errorTf])
sec4 = ipw.HBox([plotDis])
app = display(sec1,sec2,out,sec3,sec4)
app

Output()

In [14]:
len(tf)

1

In [11]:
errorTf.value

60

In [16]:
interval

60

In [26]:
tf

[[[[['2020', '06', '12', '', '00', '00', '01']]],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [[['2020', '06', '12', '', '10', '01', '43']]],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [[['2020', '06', '12', '', '20', '01', '48']]],
  [],
  [],
  []]]